# Homework #3 - Labeled Faces in the Wild

Matriculation Numbers: A0124772E, A0136070R, A0121299A

Email Addresses: a0124772@u.nus.edu, e0005572@u.nus.edu, e0008742@u.nus.edu

## Boilerplate

In [1]:
###################
##### IMPORTS #####
###################

# Standard Library
import math

# Numpy
import numpy as np

# SciKit
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report

# Image Processing
import dlib
import cv2


In [2]:
#####################################
##### MODEL SELECTION FRAMEWORK #####
#####################################

class LearningModel():
    def __init__(self):
        self.X              = None    # original points
        self.y              = None    # original classifications
        self.trained_models = None
        self.best_model     = None
        
    
    def supplyDataset(self, all_samples, all_labels):
        '''
            Ensures there are as many labels as samples.
            
            Sets the training samples X.
            Sets the training labels y.
        '''
        assert all_samples.shape[0] == all_labels.shape[0]
        
        self.X = all_samples
        self.y = all_labels
    
    def trainAll(self, cross_validation_param=10):
        '''
            For each split of training and test sets (using k-fold),
                * train the model on the training set
                * compute E_in
                * compute E_out
                * compute F1_in
                * compute F1_out
            Store a list of the (model, E_in, E_out, F1_in, F1_out) tuples.
        '''
        self.preprocess()
        
        kf = KFold(n_splits=cross_validation_param)
        self.trained_models = []
        
        for train_idx, test_idx in kf.split(self.X):
            trained_model = self.train(self.X[train_idx], self.y[train_idx])
            
            E_in   = self.getError(trained_model, self.X[train_idx], self.y[train_idx])
            E_out  = self.getError(trained_model, self.X[test_idx], self.y[test_idx])
            F1_in  = self.getF1(trained_model, self.X[train_idx], self.y[train_idx])
            F1_out = self.getF1(trained_model, self.X[test_idx], self.y[test_idx])
            
            self.trained_models.append((trained_model, E_in, E_out, F1_in, F1_out))
        
        self.postprocess()
    
    def getAverageErrors(self):
        '''
            Returns a pair of the in-sample error and the average out-of-sample error.
        '''
        sum_E_in  = 0
        sum_E_out = 0
        
        for model in self.trained_models:
            E_in  = model[1]
            E_out = model[2]
            
            sum_E_in                 += E_in
            sum_E_out                += E_out
        
        average_E_in                 = sum_E_in                 / len(self.trained_models)
        average_E_out                = sum_E_out                / len(self.trained_models)
        
        return (average_E_in, average_E_out)
    
    def getAverageF1(self):
        '''
            Returns the F1 scores of all the models on the data designated as output (i.e. F1_out values)
        '''
        f1s = []
        
        for model in self.trained_models:
            f1s.append(model[4])
        
        return np.mean(f1s)
    
    def getError(self, classifier, points, classifications):
        '''
            Calculate the error of a model over a label given a sample dataset and labels for it.
            0/1-loss is used as this is a classification problem.
        '''

        # use the model to predict the classifications of all points in the test set
        predicted_classifications = classifier.predict(points)

        # calculate the error using 0/1 loss
        N = predicted_classifications.shape[0]
        assert N == classifications.shape[0]
        num_misclassifications = 0
        for i in range(0, N):
            if predicted_classifications[i] != classifications[i]:
                num_misclassifications += 1

        return num_misclassifications/N
    
    def getF1(self, classifier, points, classifications):
        '''
            Compute the F1 score for specified model.
        '''
        predicted_classifications = classifier.predict(points)
        return f1_score(classifications, predicted_classifications, average='micro')
    
    def getBestModel(self):
        '''
            Returns the best model by F1_out score.
        '''
        best_model, best_score = None, 0
        
        for model in self.trained_models:
            if model[4] > best_score:
                best_model, best_score = model, model[4]
        
        return best_model
    
    def removeJunkModels(self):
        '''
            Wipes out self.trained_models, puts the best model back in.
            Do this if we are sure we only want to keep the best models.
            All predictions, etc, will only use a single model after this is done.
        '''
        self.trained_models = [self.getBestModel()]
    
    def predict(self, points):
        '''
            Get the modal prediction across all the models.
        '''
        num_points = points.shape[0]
        predictions_by_model = [] # ith element is prediction of all points by model i
        predictions_by_point = [] # ith element is modal prediction of point i by all models
        
        for model in self.trained_models:
            predictions_by_model.append(model[0].predict(points))
        
        predictions_by_model = np.array(predictions_by_model)
        
        for i in range(0, num_points):
            point_i_modal_prediction = np.argmax(np.bincount(predictions_by_model[:, i]))
            predictions_by_point.append(point_i_modal_prediction)
            
        predictions_by_point = np.array(predictions_by_point)
        
        return predictions_by_point
    
    def preprocess(self):
        pass
    
    def train(self, points, classifications):
        pass
    
    def postprocess(self):
        pass


class ScalingPCA_Classifier():
    '''
        Encapsulates models which use scaling followed by PCA.
        Allows predictions using the model.
    '''
    
    def __init__(self, classifier, scaler, pca):
        self.classifier = classifier
        self.scaler     = scaler
        self.pca        = pca
    
    def predict(self, points):
        return self.classifier.predict(self.pca.transform(self.scaler.transform(points)))


In [3]:
######################################
##### IMAGE PROCESSING FUNCTIONS #####
######################################

def getAlignedImage(X):
    '''
        Computes an aligned image given an original image.
        Adapted from
    '''
    
    # get file from http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
    # unzip from bz2
    predictor_model = "shape_predictor_68_face_landmarks.dat"
    face_detector = dlib.get_frontal_face_detector()
    face_pose_predictor = dlib.shape_predictor(predictor_model)

    h, w = 50, 37

    # from http://www.learnopencv.com/average-face-opencv-c-python-tutorial/

    # Compute similarity transform given two sets of two points.
    # OpenCV requires 3 pairs of corresponding points.
    # We are faking the third one.

    def similarityTransform(inPoints, outPoints) :
        s60 = math.sin(60*math.pi/180);
        c60 = math.cos(60*math.pi/180);  
        inPts = np.copy(inPoints).tolist();
        outPts = np.copy(outPoints).tolist();
        xin = c60*(inPts[0][0] - inPts[1][0]) - s60*(inPts[0][1] - inPts[1][1]) + inPts[1][0];
        yin = s60*(inPts[0][0] - inPts[1][0]) + c60*(inPts[0][1] - inPts[1][1]) + inPts[1][1];
        inPts.append([np.int(xin), np.int(yin)]);
        xout = c60*(outPts[0][0] - outPts[1][0]) - s60*(outPts[0][1] - outPts[1][1]) + outPts[1][0];
        yout = s60*(outPts[0][0] - outPts[1][0]) + c60*(outPts[0][1] - outPts[1][1]) + outPts[1][1];
        outPts.append([np.int(xout), np.int(yout)]);
        tform = cv2.estimateRigidTransform(np.array([inPts]), np.array([outPts]), False);
        return tform;

    def get_points(flat_images, h=h, w=w):
        points = []
        for i in range(len(flat_images)):
            image = np.uint8(flat_images[i].reshape((h,w)))
            points.append([])
            detected_faces = face_detector(image, 1)
            for j, face_rect in enumerate(detected_faces):
                pose_landmarks = face_pose_predictor(image, face_rect)
                for k in range(68): 
                    point = dlib.full_object_detection.part(pose_landmarks, k)
                    points[i].append((point.x, point.y))
                break
        return points

    # adapted from http://www.learnopencv.com/average-face-opencv-c-python-tutorial/
    # align eye corners at 0.3h, 0.15w/0.85w
    def align_eyes(allPoints, images, h=h, w=w, n=68):
        # Eye corners
        eyecornerDst = [ (np.int(0.15 * w ), np.int(h / 3)), (np.int(0.85 * w ), np.int(h / 3)) ];
        imagesNorm = []; 
        numImages = len(images)
        numCount = 0
        # Warp images and trasnform landmarks to output coordinate system,
        # and find average of transformed landmarks.  
        for i in range(numImages):
            # dont change image if no face detected
            if len(allPoints[i]) == 0:
                imagesNorm.append(images[i])
                continue;
            # Corners of the eye in input image
            eyecornerSrc  = [ allPoints[i][36], allPoints[i][45] ] ;
            # Compute similarity transform
            tform = similarityTransform(eyecornerSrc, eyecornerDst);
            # Apply similarity transformation
            img = cv2.warpAffine(images[i], tform, (w,h));
            # Calculate location of average landmark points.
            imagesNorm.append(img);
        return imagesNorm

    def to_image(flat_images):
        return [img.reshape((h,w)) for img in flat_images]

    def image_to_np_array(images, h=h, w=w):
        array = np.zeros((len(images), h*w))
        for k in range(len(images)):
            for i in range(h):
                for j in range(w):
                    array[k][i*37+j] = images[k][i][j]
        return array

    X_points = get_points(X)
    X_images_norm = align_eyes(X_points, to_image(X))
    X_align = image_to_np_array(X_images_norm)
    
    return X_align


In [4]:
class ScalingPCA_NeuralNet(LearningModel):
    def predict(self, points):
        return super().predict(getAlignedImage(points))
    
    def preprocess(self):
        self.X = getAlignedImage(self.X)
    
    def train(self, points, classifications):
        scaler = StandardScaler()
        scaler.fit(points)
        
        points_scaled = scaler.transform(points)
        
        pca = PCA(n_components=150, svd_solver='randomized', whiten=True)
        pca.fit(points_scaled)
        
        points_scaled_pca = pca.transform(points_scaled)
        
        param_grid = {'solver': ['adam','lbfgs'], 'alpha': [1e-1, 1e-2, 1e-3]}
        clf = GridSearchCV(MLPClassifier(), param_grid, scoring='f1_micro')
        
        clf = clf.fit(points_scaled_pca, classifications)
        return ScalingPCA_Classifier(clf, scaler, pca)


X = np.load('X_train.npy')
y = np.load('y_train.npy')

neuralnet = ScalingPCA_NeuralNet()

neuralnet.supplyDataset(X, y)
neuralnet.trainAll()
neuralnet.removeJunkModels()

print(neuralnet.getAverageF1())


def print_labels_to_file(filename, labels):
    fo = open(filename,'w')
    fo.write('ImageId,PredictedClass\n')
    for i in range(labels.shape[0]):
        fo.write(str(i) + ',' + str(labels[i])+'\n')
    fo.close()

test_set = np.load('X_test.npy')

print_labels_to_file('transform_pca_nn.csv', neuralnet.predict(test_set))


0.917525773196


In [ ]:
class SimpleSVM(LearningModel):
    def train(self, points, classifications):
        param_grid = {'C': [10**p for p in range(-2, 3)],
                      'kernel': ['linear', 'poly'],
                      'gamma': [1e-1, 1e-2, 1e-3],
                      'degree': list(range(2, 5))}
        clf = GridSearchCV(SVC(), param_grid, scoring='f1_micro')
        
        clf = clf.fit(points, classifications)
        return clf

X = np.load('X_train.npy')
y = np.load('y_train.npy')

svm = SimpleSVM()

svm.supplyDataset(X, y)
svm.trainAll()
svm.removeJunkModels()

print(svm.getAverageF1())


## Statement of (Team-Level) Individual Work

Please initial (between the square brackets) one of the following statements.

[X] I, A0124772E and A0136070R and A0121299A, certify that we have followed the CS 3244 Machine Learning class guidelines for homework assignments.  In particular, we expressly vow that we have followed the Facebook rule in discussing with others (out of our team) in doing the assignment and did not take notes (digital or printed) from the discussions.  

[ ] I, <*substitute your matric number here*>, did not follow the class rules regarding the homework assignment, because of the following reason:

<*Please fill in*>

I suggest that I should be graded as follows:

<*Please fill in*>

### References

